# 🧠 3) Embeddings locaux avec Ollama + Stockage vectoriel avec FAISS (Meta)

On utilise **mxbai-embed-large** :

- modèle open-source
- rapide
- optimisé CPU
- 1024 dimensions

➡ Chaque chunk est transformé en vecteur numérique.

Grâce à ça, le système peut mesurer la similarité conceptuelle, pas juste les mots.

On utilise **FAISS**, la technologie de Meta pour :

- stocker les embeddings
- effectuer des recherches rapides ~O(log n)
- gérer des milliers de docs

👉 Objectifs atteint : 
- `conversion du savoir ESILV en vecteurs utilisables par un moteur de recherche sémantique.`
- `vector store professionnel utilisé dans les systèmes RAG modernes.`

In [1]:
import os
import json
import faiss
import numpy as np
import ollama

In [2]:
# Fonction d’embedding (Ollama)

def embed_text(text):
    resp = ollama.embeddings(
        model="mxbai-embed-large",
        prompt=text
    )
    return np.array(resp["embedding"], dtype="float32")

In [3]:
# Load FAISS ou créer un nouvel index

INDEX_PATH = "vector_store/faiss_index.bin"
MAPPING_PATH = "vector_store/mapping.json"

# Dimensions embed (mxbai-embed-large = 1024)
EMBED_DIM = 1024  

def load_or_create_faiss():
    if os.path.exists(INDEX_PATH):
        index = faiss.read_index(INDEX_PATH)
        print("Index FAISS chargé.")
    else:
        index = faiss.IndexFlatL2(EMBED_DIM)
        print("Nouvel index FAISS créé.")

    # mapping id -> infos
    if os.path.exists(MAPPING_PATH):
        with open(MAPPING_PATH, "r", encoding="utf-8") as f:
            mapping = json.load(f)
    else:
        mapping = {}

    return index, mapping

In [4]:
# Fonction : encoder UN fichier chunk JSON

def embed_chunk_file(filepath, index, mapping):
    print(f"➡ Traitement de : {filepath}")

    with open(filepath, "r", encoding="utf-8") as f:
        chunks = json.load(f)

    for chunk in chunks:
        text = chunk.get("content", "")
        if not text.strip():
            continue

        vec = embed_text(text).reshape(1, -1)

        # ajouter à FAISS
        idx_id = index.ntotal
        index.add(vec)

        # enregistrer le mapping
        mapping[str(idx_id)] = {
            "title": chunk.get("title"),
            "content": chunk.get("content"),
            "rubric": chunk.get("rubric"),
            "url": chunk.get("url"),
            "source_file": os.path.basename(filepath)
        }

    print(f"✔ Embeddings ajoutés depuis : {filepath}")

In [5]:
# Sauvegarde automatique

def save_all(index, mapping):
    faiss.write_index(index, INDEX_PATH)
    with open(MAPPING_PATH, "w", encoding="utf-8") as f:
        json.dump(mapping, f, indent=2, ensure_ascii=False)
    print("💾 FAISS + mapping sauvegardés.")

In [23]:
os.listdir("../../data/chunks_esilv")

['chunks_admissions.json',
 'chunks_entreprises-debouches.json',
 'chunks_formations.json',
 'chunks_international.json',
 'chunks_lecole.json',
 'chunks_recherche.json']

In [24]:
# Pipeline complet : fichier par fichier

def run_embedding_pipeline():
    index, mapping = load_or_create_faiss()

    for fn in os.listdir("../../data/chunks_esilv"):
        if not fn.endswith(".json"):
            continue

        embed_chunk_file(
            filepath=os.path.join("../../data/chunks_esilv", fn),
            index=index,
            mapping=mapping
        )

        save_all(index, mapping)

    print("\n🎉 Tous les embeddings ont été générés et stockés dans FAISS.")

In [25]:
run_embedding_pipeline()

Nouvel index FAISS créé.
➡ Traitement de : ../../data/scraping/chunks_esilv\chunks_admissions.json
✔ Embeddings ajoutés depuis : ../../data/scraping/chunks_esilv\chunks_admissions.json
💾 FAISS + mapping sauvegardés.
➡ Traitement de : ../../data/scraping/chunks_esilv\chunks_entreprises-debouches.json
✔ Embeddings ajoutés depuis : ../../data/scraping/chunks_esilv\chunks_entreprises-debouches.json
💾 FAISS + mapping sauvegardés.
➡ Traitement de : ../../data/scraping/chunks_esilv\chunks_formations.json
✔ Embeddings ajoutés depuis : ../../data/scraping/chunks_esilv\chunks_formations.json
💾 FAISS + mapping sauvegardés.
➡ Traitement de : ../../data/scraping/chunks_esilv\chunks_international.json
✔ Embeddings ajoutés depuis : ../../data/scraping/chunks_esilv\chunks_international.json
💾 FAISS + mapping sauvegardés.
➡ Traitement de : ../../data/scraping/chunks_esilv\chunks_lecole.json
✔ Embeddings ajoutés depuis : ../../data/scraping/chunks_esilv\chunks_lecole.json
💾 FAISS + mapping sauvegardés.


In [26]:
# Tester la similarité (retrieval)

def search(query, k=5):
    index, mapping = load_or_create_faiss()
    q_vec = embed_text(query).reshape(1, -1)
    D, I = index.search(q_vec, k)

    results = []
    for dist, idx in zip(D[0], I[0]):
        results.append({
            "score": float(dist),
            "title": mapping[str(idx)]["title"],
            "content": mapping[str(idx)]["content"][:300] + "...",
            "url": mapping[str(idx)]["url"]
        })

    return results

In [27]:
search("Comment intégrer l'ESILV après un bac+2 ?", k=3)

Index FAISS chargé.


[{'score': 149.9091339111328,
  'title': 'admissions – Introduction',
  'content': 'Les deux flux principaux d’intégration à l’ESILV sont le Concours AVENIR BAC en admission post-bac via la procédure Parcoursup, et le concours AVENIR PREPAS après une classe préparatoire aux grandes écoles.. Hors ces procédures spécifiques aux étudiants de Terminale Générale et CPGE, vous pouvez int...',
  'url': 'https://www.esilv.fr/admissions/'},
 {'score': 162.86717224121094,
  'title': 'Concours AVENIR BAC 2026 – Full text',
  'content': 'Sur Parcoursup, pour l’ESILV, vous devez sélectionner les voeux intitulés (en fonction de vos spécialités) : Formation d’ingénieur Bac + 5 – Bac série générale avec la spécialité Maths en Terminale + une spécialité scientifique Formation d’ingénieur Bac + 5 – Bac Série générale avec la spécialité Ma...',
  'url': 'https://www.esilv.fr/admissions/concours-avenir/'},
 {'score': 177.98876953125,
  'title': 'Acquérir des fondamentaux dans les domaines du numérique et 